In [2]:
import config
from dataloader import iMetDataset
import pandas as pd

In [3]:
conf = config.Config()
train_df = pd.read_csv(conf.DATA_PATH+'/train.csv')

# Model

In [4]:
import torch
import torchvision
import torch.nn as nn
import copy
import utils

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False, progress=True, num_classes=46, pretrained_backbone=True)
myModel = copy.deepcopy(model)

In [5]:
d = iMetDataset(conf, train_df)
#dataset = dataloader.iMetDataset(conf, train_df, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True, num_workers=4)
print(data_loader)


# The model has 4 main parts
<ol>
<li>transform(GeneralizedRCNNTransform)</li> 
<li>backbone(BackbonewithFPN and FPN-FeaturePyramidNetwork)</li>
<li>rpn (RegionProposalNetwork)</li>
<li>roi_heads - RoIHeads</li>
</ol>

We need to change roi_heads as it has TwoMLPHead, FastRCNNPredictor, MaskRCNNHeads, MaskRCNNPredictor ()

In [7]:
import sys
sys.path.append("/path/to/file/") 

from vision.references.detection.engine import train_one_epoch, evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    d = iMetDataset(conf, train_df)
    data_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True, num_workers=4)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
    
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3, gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

ModuleNotFoundError: No module named 'coco_utils'